In [1]:
import os
import pyterrier as pt
import pyterrier_rag as ptr
from datasets import load_dataset
import ir_datasets
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np
import pyterrier_alpha as pta

/opt/miniconda3/envs/emalir-dr-rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ----------------------------------------------------------------------------
# 2. Caricamento dataset GPQA
# ----------------------------------------------------------------------------
print("🔍 Carico GPQA da Hugging Face...")
mmlu = load_dataset("Idavidrein/gpqa", "gpqa_diamond", split="train")

🔍 Carico GPQA da Hugging Face...


Using the latest cached version of the dataset since Idavidrein/gpqa couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'gpqa_diamond' at /root/.cache/huggingface/datasets/Idavidrein___gpqa/gpqa_diamond/0.0.0/90b8e5be2b1d3d2dbfe016cdab47981150600c4a (last modified on Sat Jul 26 13:40:15 2025).


In [3]:
# Build a DataFrame from the dataset with 'query' and 'gold_answer' columns
df = pd.DataFrame(mmlu)
df = df.rename(columns={"Record ID": "qid","Question": "query", "Correct Answer": "gold_answer"})
df = df[["qid", "query", "gold_answer"]]

---

In [6]:
data = df.rename(columns={"qid": "qid", "query": "text"})
data = data[['qid', 'text']]
data["text"] = data["text"].str.replace('\n', '\\n')
data.to_csv("../data/raw/rag/gpqa-queries.tsv", sep="\t", index=False)

In [4]:
dataset = ir_datasets.load('msmarco-segment-v2.1')
pt_dataset = pt.get_dataset("irds:msmarco-segment-v2.1")

In [5]:
def rename_segment(run):
    run = run.rename(columns={"segment": "text"})
    return run
rename_pipe = pt.apply.generic(rename_segment)

In [6]:
index = pta.Artifact.from_hf('namawho/msmarco-segment-v2.1.pisa')
bm25_ret = index.bm25(verbose=True) % 200 >> pt.text.get_text(pt_dataset, "segment") >> rename_pipe

In [7]:
retrieved = bm25_ret.transform(df)

PISA bm25: 100%|██████████| 198/198 [07:17<00:00,  2.21s/query]


In [8]:
retrieved["Q0"] = "Q0"
retrieved["tag"] = "bm25"
retrieved = retrieved[["qid","Q0","docno","rank","score","tag"]]

# scrivi su file
retrieved.to_csv("../data/raw/rag/__bm25__msmarco-segment-gpqa.run", sep=" ", index=False, header=False)

---

In [22]:
print("🔍 Carico ranking esistente...")
df_run_base = pd.read_csv(
    "../data/processed/rag/__setencoder-novelty-base__msmarco-segment-gpqa.tsv", sep="\t",
    names=["qid", "Q0", "docno", "rank", "score", "run_name", "text"]
)
df_run_ea = pd.read_csv(
    "../data/processed/rag/__setencoder-novelty-ea__msmarco-segment-gpqa.tsv", sep="\t",
    names=["qid", "Q0", "docno", "rank", "score", "run_name", "text"]
)

🔍 Carico ranking esistente...


In [5]:
dataset = ir_datasets.load('msmarco-segment-v2.1')
pt_dataset = pt.get_dataset("irds:msmarco-segment-v2.1")
#total_docs = dataset.docs_count() 
#all_docs = list(tqdm(dataset.docs_iter(), total=total_docs))

In [6]:
import pyterrier_alpha as pta
index = pta.Artifact.from_hf('namawho/msmarco-segment-v2.1.pisa')

In [7]:
#doc_dict = {doc.doc_id: doc for doc in all_docs}

In [8]:
from pyterrier_rag.backend import OpenAIBackend
from pyterrier_rag.prompt import Concatenator
from pyterrier_rag.readers import Reader
from pyterrier_rag.prompt import PromptTransformer, prompt
from fastchat.model import get_conversation_template

In [9]:
#model_name = "llama-3-8b-instruct"
model_name = "llama-3.3-70b-instruct"

In [10]:
system_message = r"""You are an expert Q&A system that is trusted around the world. 
        Always answer the query using the provided context information, and not prior knowledge.

        Some rules to follow:
        1. Never directly reference the given context in your answer
        2. Avoid statements like 'Based on the context, ...' or 
        'The context information ...' or anything along those lines."""
prompt_text = """Context information is below.
            ---------------------
            {{ qcontext }}
            ---------------------
            Given the context information, answer to the given query.
            
            Query: {{ query }}

            Do not include any explanation or additional text.

            Answer: """

template = get_conversation_template("meta-llama-3.1-sp")
prompt = PromptTransformer(
    conversation_template=template,
    system_message=system_message,
    instruction=prompt_text,
    input_fields=["query", "qcontext"],
    api_type="openai"
)

In [20]:
def get_rank(df_queries):
    run = df_queries.merge(df_run_ea, on="qid", how="left")
    return run
get_rank_pipe = pt.apply.generic(get_rank)

def rename_segment(run):
    run = run.rename(columns={"segment": "text"})
    return run
rename_pipe = pt.apply.generic(rename_segment)

In [12]:
from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("casperhansen/llama-3-8b-instruct-awq")
tokenizer = AutoTokenizer.from_pretrained("casperhansen/llama-3.3-70b-instruct-awq")

generation_args={
    "temperature": 0.6,
    "max_tokens": 140,
}

# this could equally be a real OpenAI models
llama = OpenAIBackend(model_name, 
                      api_key=os.environ['IDA_LLM_API_KEY'],
                      generation_args=generation_args,
                      base_url="http://api.llm.apps.os.dcs.gla.ac.uk/v1", 
                      verbose=True, 
                      parallel=64)

In [13]:
llama_reader = Reader(llama, prompt=prompt)
set_encoder_llama = get_rank_pipe % 5 >>  pt.text.get_text(pt_dataset, "segment") >> rename_pipe >> Concatenator(tokenizer=tokenizer, max_length=8191,max_per_context=1638) >> llama_reader

In [14]:
set_encoder_llama.transform(df.head(1))

/opt/miniconda3/envs/emalir-dr-rag/lib/python3.10/site-packages/pyterrier/transformer.py:101: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return pd.DataFrame(list(self.transform_iter(inp.to_dict(orient='records'))))


,prompt,qid,query_0,qanswer
0,"[{'role': 'system', 'content': 'You are an exp...",rec06pnAkLOr2t2mp,Two quantum states with energies E1 and E2 hav...,ΔE = 6.58 * 10^-4 eV


In [ ]:
from tqdm import tqdm

save_every=100
partial_save_path="../data/processed/rag/ea_gpqa_rag_output_cut_5.tsv"

try:
    df_partial = pd.read_csv(partial_save_path, sep="\t")
    done_qids = set(df_partial["qid"])
    print(f"✅ Ripresi {len(done_qids)} risultati da salvataggio parziale.")
    results = [df_partial]
except FileNotFoundError:
    print("🚨 Nessun salvataggio trovato, si parte da zero.")
    done_qids = set()
    results = []

remaining = df[~df["qid"].isin(done_qids)].reset_index(drop=True)
print(f"🧠 Da processare: {len(remaining)} esempi.")
    
for row in tqdm(remaining.iterrows(), total=len(remaining), desc="🔁 RAG on MMLU"):
    idx, data = row
    result = set_encoder_llama.transform(pd.DataFrame([data]))

    result_merged = result.merge(df[["qid", "gold_answer"]], on="qid", how="left")
    results.append(result_merged)

    # ✅ Salvataggio intermedio
    if (idx + 1) % save_every == 0 or (idx + 1) == len(remaining):
        df_save = pd.concat(results, ignore_index=True)
        df_save.to_csv(partial_save_path, sep="\t", index=False)
        print(f"💾 Salvati {len(df_save)} risultati su {partial_save_path}")
        results = [df_save]

🚨 Nessun salvataggio trovato, si parte da zero.
🧠 Da processare: 198 esempi.


🔁 RAG on MMLU:  51%|█████     | 100/198 [03:54<01:55,  1.17s/it]

💾 Salvati 100 risultati su ../data/processed/rag/ea_gpqa_rag_output_cut_5.tsv


🔁 RAG on MMLU:  84%|████████▍ | 166/198 [07:10<02:14,  4.20s/it]

In [34]:
pt.Experiment(
    [set_encoder_llama],
    df[['qid', 'query']], # NB: remove .head() to run on all dev topics
    df[['qid', 'gold_answer']],
    [ptr.measures.F1, ptr.measures.EM],
    batch_size=25,
    verbose=True,
    names=['set encoder'],
)

pt.Experiment: 100%|██████████| 8/8 [05:35<00:00, 41.99s/batches]


,name,F1,EM
0,set encoder,0.127162,0.035354


# Score@3

In [ ]:
# name	F1	EM
# set encoder base	0.134688	0.040404

In [ ]:
# name	F1	EM
# 0	set encoder ea	0.133429	0.040404

# Score@5

In [ ]:
# 	name	F1	EM
#   set encoder	base 0.128475	0.035354


In [ ]:
# name	F1	EM
# 0	set encoder ea	0.127162	0.035354

---

In [ ]:
# test statistici

In [8]:
import pandas as pd
import pyterrier as pt
import pyterrier_rag as ptr
from datasets import load_dataset
import ir_datasets
import pyterrier_alpha as pta
from pyterrier_rag.backend import OpenAIBackend
from pyterrier_rag.prompt import Concatenator
from pyterrier_rag.readers import Reader
from pyterrier_rag.prompt import PromptTransformer, prompt
from fastchat.model import get_conversation_template
import os 

print("🔍 Carico GPQA da Hugging Face...")
mmlu = load_dataset("Idavidrein/gpqa", "gpqa_diamond", split="train")
df = pd.DataFrame(mmlu)
df = df.rename(columns={"Record ID": "qid","Question": "query", "Correct Answer": "gold_answer"})
df = df[["qid", "query", "gold_answer"]]

df_output_base = pd.read_csv("../data/processed/rag/base_gpqa_rag_output_cut_5.tsv", sep="\t")
df_output_ea   = pd.read_csv("../data/processed/rag/ea_gpqa_rag_output_cut_5.tsv",   sep="\t")

df_run_base = pd.read_csv("../data/processed/rag/__setencoder-novelty-base__msmarco-segment-gpqa.tsv", sep="\t",
    names=["qid", "Q0", "docno", "rank", "score", "run_name", "text"]
)
df_run_ea = pd.read_csv("../data/processed/rag/__setencoder-novelty-ea__msmarco-segment-gpqa.tsv", sep="\t",
    names=["qid", "Q0", "docno", "rank", "score", "run_name", "text"]
)

dataset = ir_datasets.load('msmarco-segment-v2.1')
pt_dataset = pt.get_dataset("irds:msmarco-segment-v2.1")
index = pta.Artifact.from_hf('namawho/msmarco-segment-v2.1.pisa')

model_name = "llama-3.3-70b-instruct"
system_message = r"""You are an expert Q&A system that is trusted around the world. 
        Always answer the query using the provided context information, and not prior knowledge.

        Some rules to follow:
        1. Never directly reference the given context in your answer
        2. Avoid statements like 'Based on the context, ...' or 
        'The context information ...' or anything along those lines."""
prompt_text = """Context information is below.
            ---------------------
            {{ qcontext }}
            ---------------------
            Given the context information, answer to the given query.
            
            Query: {{ query }}

            Do not include any explanation or additional text.

            Answer: """

template = get_conversation_template("meta-llama-3.1-sp")
prompt = PromptTransformer(
    conversation_template=template,
    system_message=system_message,
    instruction=prompt_text,
    input_fields=["query", "qcontext"],
    api_type="openai"
)

def rename_segment(run):
    run = run.rename(columns={"segment": "text"})
    return run
rename_pipe = pt.apply.generic(rename_segment)

def get_rank_base(df_queries):
    run = df_queries.merge(df_run_base, on="qid", how="left")
    return run
get_rank_base_pipe = pt.apply.generic(get_rank_base)

def get_rank_ea(df_queries):
    run = df_queries.merge(df_run_ea, on="qid", how="left")
    return run
get_rank_ea_pipe = pt.apply.generic(get_rank_ea)

def get_output_base(df_queries):
    print(df_output_base.head(1))
    return df_output_base
get_output_base_pipe = pt.apply.generic(get_output_base)

def get_output_ea(df_queries):
    return df_output_ea
get_output_ea_pipe = pt.apply.generic(get_output_ea)

from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("casperhansen/llama-3-8b-instruct-awq")
tokenizer = AutoTokenizer.from_pretrained("casperhansen/llama-3.3-70b-instruct-awq")

generation_args={
    "temperature": 0.6,
    "max_tokens": 140,
}

# this could equally be a real OpenAI models
llama = OpenAIBackend(model_name, 
                      api_key=os.environ['IDA_LLM_API_KEY'],
                      generation_args=generation_args,
                      base_url="http://api.llm.apps.os.dcs.gla.ac.uk/v1", 
                      verbose=True, 
                      parallel=64)
llama_reader = Reader(llama, prompt=prompt)

pipe_base = get_rank_base_pipe % 5 >>  pt.text.get_text(pt_dataset, "segment") >> rename_pipe >> Concatenator(tokenizer=tokenizer, max_length=8191,max_per_context=1638) >> llama_reader
pipe_ea = get_rank_ea_pipe % 5 >>  pt.text.get_text(pt_dataset, "segment") >> rename_pipe >> Concatenator(tokenizer=tokenizer, max_length=8191,max_per_context=1638) >> llama_reader

🔍 Carico GPQA da Hugging Face...


In [9]:
from statsmodels.stats.contingency_tables import mcnemar
import numpy as np

def mcnemar_callable(a, b):
    # a,b = array per-query di EM (0/1) dei due modelli, stesso ordine di qid
    a = np.asarray(a).astype(int)
    b = np.asarray(b).astype(int)
    # Discordanti:
    only_a = np.sum((a == 1) & (b == 0))
    only_b = np.sum((a == 0) & (b == 1))
    table = [[0, only_a],
             [only_b, 0]]
    res = mcnemar(table, exact=False, correction=True)
    # PyTerrier si aspetta (stat, pvalue)
    return (res.statistic, res.pvalue)

pt.Experiment(
    [pipe_base, pipe_ea],
    df[['qid', 'query']],
    df[['qid', 'gold_answer']],
    [ptr.measures.EM],
    batch_size=25,
    verbose=True,
    baseline=0, 
    test=mcnemar_callable, 
    correction="holm",
    names=['set encoder base', 'set encoder ea'],
)

pt.Experiment:   0%|          | 0/16 [00:00<?, ?batches/s]/opt/miniconda3/envs/emalir-dr-rag/lib/python3.10/site-packages/pyterrier/transformer.py:101: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return pd.DataFrame(list(self.transform_iter(inp.to_dict(orient='records'))))
pt.Experiment:   6%|▋         | 1/16 [01:38<24:43, 98.92s/batches]/opt/miniconda3/envs/emalir-dr-rag/lib/python3.10/site-packages/pyterrier/transformer.py:101: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return pd.DataFrame(list(self.transform_iter(inp.to_dict(orient='records'))))
pt.Experiment:  12%|█▎        | 2/16 [03:14<22:38, 97.07s/batches]/opt/miniconda3/envs/emalir-dr-rag/lib/python3.10/site-packages/pyterrier/transformer.py:101: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return pd.DataFrame(list(self.transform_iter(inp.to_dict(orient='records'))))
pt.Experiment:  19%|█▉        | 3/16 [04:57<21:35, 99.66s

,name,EM,EM +,EM -,EM p-value,EM reject,EM p-value corrected
0,set encoder base,0.025253,NaN,NaN,NaN,False,NaN
1,set encoder ea,0.030303,1.0,0.0,1.0,False,1.0
